## Training plan:
Using LLama Llama-3.2-1B-bnb-4bit (not instruct model)
we train 2 variations:

We train one epoch with 94.5 of the data with 5% val 0.5% test. We also include our new data. : https://huggingface.co/datasets/mlfoundations-dev/stackexchange_mythology/viewer/default/train?p=6 ?

Where 90% train, 10 % test

For our own data, we dupplicate training data

- One with new data same hyperparams
- One with new data and better hyperparam



In [1]:
%%capture
%uv pip install unsloth
# Also get the latest nightly Unsloth!
%uv pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)



from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Wed Dec  3 17:28:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.57.08              Driver Version: 575.57.08      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  |   00000000:90:1D.0 Off |                    0 |
| N/A   22C    P0             50W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
print(gpu_info)

Wed Dec  3 17:28:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.57.08              Driver Version: 575.57.08      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          On  |   00000000:90:1D.0 Off |                    0 |
| N/A   22C    P0             50W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
from peft import PeftModel
from unsloth import FastLanguageModel

import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           #! MODEL WE USE!
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

"""model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/granite-4.0-h-1b-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)"""


def load_original():

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/Qwen2-0.5B-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
        # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
    )

    model = FastLanguageModel.get_peft_model(
      model,
      r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
      target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj",],
      lora_alpha = 16, # kinda like lr
      lora_dropout = 0, # Supports any, but = 0 is optimized
      bias = "none",    # Supports any, but = "none" is optimized


      use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
      random_state = 3407,
      use_rslora = False,  # We support rank stabilized LoRA
      loftq_config = None, # And LoftQ
  )



    return model, tokenizer


def load_new():

    org_model, org_tokenizer = FastLanguageModel.from_pretrained(
          model_name = "unsloth/granite-4.0-h-1b-bnb-4bit", # or choose "unsloth/Llama-3.2-1B-Instruct"
          max_seq_length = max_seq_length,
          dtype = dtype,
          load_in_4bit = load_in_4bit,
          # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
      )

    model = PeftModel.from_pretrained(
    org_model,
    "StefanCoder1/Scalable___", #! Definera om nödvändigt
    token=None, )

    return model, org_tokenizer

model, tokenizer = load_original()

/tmp/ipykernel_169/2129107997.py:2: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.6: Fast Qwen2 patching. Transformers: 4.56.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.494 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 8.0. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/457M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Unsloth 2025.11.6 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",#"llama-3.2", #! CHANGED
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset


We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [7]:
from unsloth.chat_templates import standardize_sharegpt

dataset2 = load_dataset("mlfoundations-dev/stackexchange_mythology", split="train")
dataset2 = dataset2.select_columns(["conversations"])
dataset2 = standardize_sharegpt(dataset2)
dataset2 = dataset2.map(formatting_prompts_func, batched = True,)


Unsloth: Standardizing formats (num_proc=43):  91%|######### | 4810/5304 [00:00<?, ? examples/s]

Map:   0%|          | 0/5304 [00:00<?, ? examples/s]

In [9]:
from unsloth.chat_templates import standardize_sharegpt
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Unsloth: Standardizing formats (num_proc=43):  86%|########6 | 86046/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [10]:
dataset[5]["conversations"]
dataset2[5]["conversations"]

[{'content': 'I know that much of Greek mythology was passed on by word of mouth but someone must have been the first to collect this in one source.  What was that Compendium and is there a version available in modern English?\n',
  'role': 'user'},
 {'content': 'The most notable ancient work that serves as a compendium of Greek mythology is "The Library" (also known as "Bibliotheca"), attributed to Apollodorus, a Greek scholar of the 2nd century BCE. This work is one of the earliest comprehensive collections of mythological tales and genealogies of gods, heroes, and legendary figures.\n\nAnother significant source is "Metamorphoses" by Ovid, a Roman poet who lived in the 1st century CE. Although it is not strictly a Greek mythological text, it retells many Greek myths in a narrative form.\n\nIn modern English, both "The Library" of Apollodorus and Ovid\'s "Metamorphoses" have been translated and are widely available. You can find various editions of these texts, often accompanied by i

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [11]:
print(dataset[5]["text"])

print(dataset2[5]["text"])

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
How do astronomers determine the original wavelength of light emitted by a celestial body at rest, which is necessary for measuring its speed using the Doppler effect?<|im_end|>
<|im_start|>assistant
Astronomers make use of the unique spectral fingerprints of elements found in stars. These elements emit and absorb light at specific, known wavelengths, forming an absorption spectrum. By analyzing the light received from distant stars and comparing it to the laboratory-measured spectra of these elements, astronomers can identify the shifts in these wavelengths due to the Doppler effect. The observed shift tells them the extent to which the light has been redshifted or blueshifted, thereby allowing them to calculate the speed of the star along the line of sight relative to Earth.<|im_end|>

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant

In [12]:
# here we split 70/20/ 10 for train/val/test (think i have to much val )
# think 90/5/5 would be better

train_testvalid = dataset.train_test_split(test_size=0.055, seed=3407)
test_valid = train_testvalid['test'].train_test_split(test_size=1/11, seed=3407)
train_dataset = train_testvalid['train']
valid_dataset = test_valid['train']
test_dataset = test_valid['test']


print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(valid_dataset)}")
print(f"Test size: {len(test_dataset)}")

Train size: 94500
Validation size: 5000
Test size: 500


In [13]:
train_testvalid2 = dataset2.train_test_split(test_size=0.1, seed=3407)

train_dataset2 = train_testvalid2['train']

test_dataset2 = train_testvalid2['test']


print(f"Train size: {len(train_dataset2)}")
print(f"Test size: {len(test_dataset2)}")

Train size: 4773
Test size: 531


In [14]:
from datasets import concatenate_datasets

merged_train = concatenate_datasets([train_dataset, train_dataset2, train_dataset2])
merged_test = concatenate_datasets([test_dataset, test_dataset2])
merged_val = valid_dataset

print(f"Train size: {len(merged_train)}")
print(f"Test size: {len(merged_test)}")
print(f"Val size: {len(merged_val)}")

Train size: 104046
Test size: 1031
Val size: 5000


In [15]:

merged_train = merged_train.shuffle(seed=3407)
merged_test = merged_test.shuffle(seed=3407)

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
# here we ensure the HF repo exists

#%pip install huggingface_hub
from huggingface_hub import login, HfApi, create_repo



def create_HF_repo(repo_name):
    login(token=None) # dont steal it please :((((
    api = HfApi()
    user = api.whoami()["name"]
    repo_id = f"{user}/{repo_name}"
    # Ensure repo exists
    create_repo(repo_id, private=True, exist_ok=True)
    return repo_id

repo_id =create_HF_repo("Granite-tuned")


# custom callback to save lora adapters every X steps

from transformers import TrainerCallback

import os

class PushLoraCallback(TrainerCallback):
    def __init__(self, tokenizer, repo_id):
        self.tokenizer = tokenizer
        self.repo_id = repo_id
        self.version = 0
        self.api = HfApi()

    def on_save(self, args, state, control, **kwargs):
        """
        Called whenever HF Trainer writes a checkpoint.
        args.output_dir / checkpoint-{global_step}
        """
        self.version += 1

        ckpt_path = os.path.join(args.output_dir, f"checkpoint-{state.global_step}")
        subfolder = f"lora_{state.global_step}"

        print(f"\n[Callback] Uploading checkpoint from: {ckpt_path}")
        self.api.upload_folder(
            folder_path=ckpt_path,
            repo_id=self.repo_id,
        )

        # Push that EXACT model folder
        self.api.upload_folder(
            folder_path=ckpt_path,
            repo_id=self.repo_id,
            path_in_repo=subfolder,
        )

        # Optional: push logs also
        logs_subfolder = f"logs_{state.global_step}"
        self.api.upload_folder(
            folder_path="logs",
            repo_id=self.repo_id,
            path_in_repo=logs_subfolder,
        )

        print(f"Uploaded checkpoint → {subfolder}")
        return control

In [17]:
per_device_train_batch_size = 36 #!
gradient_accumulation_steps = 4

batch_size_effective = per_device_train_batch_size * gradient_accumulation_steps
steps_per_epoch = len(merged_train) // batch_size_effective
half_epoch = len(merged_train) / batch_size_effective / 2
full_epoch = len(merged_train) / batch_size_effective


logging_steps = int(steps_per_epoch/ 50)

print("Full epoch:", steps_per_epoch)
print("half epoch (steps):", half_epoch)
print("logging_steps:", logging_steps)

Full epoch: 722
half epoch (steps): 361.2708333333333
logging_steps: 14


In [18]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported



push_cb = PushLoraCallback(
    repo_id=repo_id,
    tokenizer = tokenizer

)

training_args = TrainingArguments(
        per_device_train_batch_size = per_device_train_batch_size,
        gradient_accumulation_steps = gradient_accumulation_steps,
        num_train_epochs = 2, # Set this for 1 full training run.
        learning_rate = 3e-4,  # 1 * 5 times lr
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = logging_steps,
        logging_dir = "logs",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine", # must do
        seed = 3407,
        report_to = "tensorboard",

        # saving and versioning
        output_dir = "outputs",

        #log_level = "error",          # no console logs
        log_level_replica = "error",  # no table from distributed workers
        disable_tqdm = False,         # keep the progress bar

        # for the validation data
        eval_strategy= "steps",
        eval_steps= int(steps_per_epoch // 15),

        save_strategy= "steps",
        save_steps= int(steps_per_epoch // 5),
    )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = merged_train,
    eval_dataset = valid_dataset,

    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = True, # Can make training 5x faster for short sequences.
    args = training_args,
    callbacks = [push_cb],
)

Unsloth: Tokenizing ["text"] (num_proc=47):   0%|          | 0/104046 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=47):   0%|          | 0/5000 [00:00<?, ? examples/s]

[accelerate.utils.other|WARNING]Detected kernel version 4.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [19]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user",
    response_part    = "<|im_start|>assistant",
)

Map (num_proc=47):   0%|          | 0/104046 [00:00<?, ? examples/s]

Map (num_proc=47):   0%|          | 0/5000 [00:00<?, ? examples/s]

We verify masking is actually done:

In [20]:
tokenizer.decode(trainer.train_dataset[-1]["input_ids"])

"<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nIn strong mathematical induction, the inductive hypothesis assumes the truth of P(k) for all k. Many proofs accept this assumption directly. However, why is it sometimes necessary to establish multiple base cases instead of simply assuming P(k) for the base cases, as in the example of proving that any amount of postage greater than or equal to 12 cents can be formed using 4-cent and 5-cent stamps by showing base cases up to 16 cents?<|im_end|>\n<|im_start|>assistant\nStrong induction is typically employed when dealing with recurrence relations, such as $a_{n} = a_{n-1} - a_{n-2}$. In these cases, establishing two base cases is essential to avoid logical gaps due to dependencies on previous terms (e.g., preventing an undefined $a_{-1}$ term). \n\nFor the postage stamp example, weak induction is sufficient. In weak induction, one can prove a proposition from any initial 

In [21]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

"                                                          \nThe precise degree of inclination of the sun with respect to the horizontal plane can be calculated using trigonometry and the position of the sun in the sky. The angle between the sun and the horizontal plane is known as the sun's altitude. \r\n\r\nTo calculate the sun's altitude, you will need the latitude and longitude of your location as well as the current time of day. You can then use an online calculator or formula to determine the altitude.\r\n\r\nOne formula to calculate the sun's altitude is as follows:\r\n\r\nsin(Altitude) = sin(Latitude) * sin(Declination) + cos(Latitude) * cos(Declination) * cos(Hour Angle)\r\n\r\nWhere:\r\n\r\n- Altitude is the angle between the horizon and the center of the sun in degrees\r\n- Latitude is the geographic latitude of your location in degrees\r\n- Declination is the angle between the equator and the center of the sun in degrees\r\n- Hour Angle is the difference between the local s

We can see the System and Instruction prompts are successfully masked!

In [22]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.494 GB.
0.535 GB of memory reserved.


In [23]:


trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 104,046 | Num Epochs = 2 | Total steps = 1,446
O^O/ \_/ \    Batch size per device = 36 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (36 x 4 x 1) = 144
 "-____-"     Trainable parameters = 8,798,208 of 502,830,976 (1.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
48,1.132100,1.023610
96,1.103400,1.008687
144,1.097600,0.999386
192,1.113800,0.992401
240,1.087300,0.986915
288,1.069500,0.982856
336,1.066000,0.978480
384,1.089900,0.974987
432,1.095400,0.971826
480,1.055100,0.968790


Unsloth: Not an error, but Qwen2ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient



[Callback] Uploading checkpoint from: outputs/checkpoint-144


...

  ...oint-1446/adapter_model.safetensors: 100%|##########| 35.2MB / 35.2MB            

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...out.tfevents.1764783257.modal.169.0: 100%|##########| 36.2kB / 36.2kB            

Uploaded checkpoint → lora_1446


In [24]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

10595.074 seconds used for training.
176.58 minutes used for training.
Peak reserved memory = 37.553 GB.
Peak reserved memory for training = 37.018 GB.
Peak reserved memory % of max memory = 95.085 %.
Peak reserved memory for training % of max memory = 93.731 %.


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
#model.save_pretrained("lora_model") # Local saving
#tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )